In [1]:
###   Imports and Logging   ###
import sys
import logging
import math
import copy
import numpy as np
import pandas as pd
import scipy.optimize
import matplotlib.pyplot as plt
from matplotlib import __version__ as pltver
import matplotlib.ticker as ticker
from skaero.atmosphere import coesa
from skaero import __version__ as skver
# import pint
# units = pint.UnitRegistry()


logging.basicConfig(
    level=logging.INFO,
    format=' %(asctime)s -  %(levelname)s -  %(message)s'
)

logging.info('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
logging.info('Python version: {}'.format(sys.version))
logging.info('Author: Benjamin Crews')
logging.info('Numpy version: {}'.format(np.version.version))
logging.info('Pandas version: {}'.format(pd.__version__))
logging.info('Matplotlib version: {}'.format(pltver))
logging.info('SciKit-Aero version: {}'.format(skver))
# logging.info('Pint version: {} (using standard UnitRegistry)'.format(pint.__version__))
logging.info('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')

 2020-04-07 21:37:44,761 -  INFO -  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
 2020-04-07 21:37:44,764 -  INFO -  Python version: 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
 2020-04-07 21:37:44,765 -  INFO -  Author: Benjamin Crews
 2020-04-07 21:37:44,766 -  INFO -  Numpy version: 1.16.2
 2020-04-07 21:37:44,767 -  INFO -  Pandas version: 0.24.2
 2020-04-07 21:37:44,767 -  INFO -  Matplotlib version: 3.0.3
 2020-04-07 21:37:44,767 -  INFO -  SciKit-Aero version: 0.1
 2020-04-07 21:37:44,768 -  INFO -  =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [15]:
#######################################
###   Input Rotor Characteristics   ###
#######################################
# Main Rotor
D = 35            # [ft]
R = D/2           # [ft]
b = 4             # [number of blades, no units]
CHORD = 10.4      # [in]
OMEGA = 43.2      # [rad/s]
# Rotor Solidity
sol = b*CHORD/(12*math.pi*R)
# Rotor Area
A = math.pi*R**2
# Tip Speed
v_tip = OMEGA*R   # [ft/s]


#########################################
###   Input Vehicle Characteristics   ###
#########################################
GW = 5000
# Assume a 3% download
THRUST = 1.03*GW
# Tail length to TR center of thrust
l_tail = 21.21
# Vertical tail area
S_vt = 20.92   # [ft]
# Vertical tail lift coef.
cl_vt = 0.22   # []
# Vertical tail aspect ratio
AR_vt = 3



######################################
###       Ambient Conditions       ###
######################################
alt = 0                    # [m]

atm = coesa.table(alt) # an atmosphere at altitude
# Give the atmosphere units and convert them to imperial.
h = atm[0]*3.28084         # [ft]
T = atm[1]*1.8             # [degR]
p = atm[2]/6895            # [psi]
rho = atm[3]/515           # [slug/ft3]


# Spit out the converted input to the log (and make it readable)
logging.info('')
logging.info('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
logging.info('Main Rotor Inputs:')
logging.info('   {name:>17}: {val:>7.3f} [{unit:}]'.format(name='Diameter', val=D, unit='ft'))
logging.info('   {name:>17}: {val:>7.3f} [{unit:}]'.format(name='(constant) Chord', val=CHORD, unit='in'))
logging.info('   {name:>17}: {val:>7.3f} [{unit}]'.format(name='Num Blades', val=b, unit=''))
logging.info('   {name:>17}: {val:>7.3f} [{unit:}]'.format(name='Rotational Speed', val=OMEGA, unit='rad/s'))
logging.info('   {name:>17}: {val:>7.3f} [{unit:}]'.format(name='Tip Speed', val=v_tip, unit='ft/s'))
logging.info('   {name:>17}: {val:>7.3f} [{unit:}]'.format(name='Area', val=A, unit='ft2'))
logging.info('   {name:>17}: {val:>7.3f} [{unit:}]'.format(name='Solidity', val=sol, unit=''))
logging.info('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
logging.info('Atmospheric Properties:')
logging.info('   {name:>17}: {val:>9.5f} [{unit:}]'.format(name='Altitude', val=h, unit='ft'))
logging.info('   {name:>17}: {val:>9.5f} [{unit:}]'.format(name='Temperature', val=T, unit='°F'))
logging.info('   {name:>17}: {val:>9.5f} [{unit:}]'.format(name='Pressure', val=p, unit='psi'))
logging.info('   {name:>17}: {val:>9.5f} [{unit:}]'.format(name='Density', val=rho, unit='slug/ft3'))
logging.info('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
logging.info('')

 2020-04-07 22:02:43,407 -  INFO -  
 2020-04-07 22:02:43,408 -  INFO -  -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
 2020-04-07 22:02:43,409 -  INFO -  Main Rotor Inputs:
 2020-04-07 22:02:43,409 -  INFO -              Diameter:  35.000 [ft]
 2020-04-07 22:02:43,410 -  INFO -      (constant) Chord:  10.400 [in]
 2020-04-07 22:02:43,410 -  INFO -            Num Blades:   4.000 []
 2020-04-07 22:02:43,411 -  INFO -      Rotational Speed:  43.200 [rad/s]
 2020-04-07 22:02:43,411 -  INFO -             Tip Speed: 756.000 [ft/s]
 2020-04-07 22:02:43,411 -  INFO -                  Area: 962.113 [ft2]
 2020-04-07 22:02:43,413 -  INFO -              Solidity:   0.063 []
 2020-04-07 22:02:43,416 -  INFO -  -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
 2020-04-07 22:02:43,416 -  INFO -  Atmospheric Properties:
 2020-04-07 22:02:43,417 -  INFO -              Altitude:   0.00000 [ft]
 2020-04-07 22:02:43,418 -  INFO -           Temperature: 518.67000 [°F]
 2020-04-07 22:02:43,418 -  INFO -     

In [37]:
######################################
###  Blade Downwash Distribution   ###
###           (HOGE)               ###
######################################
logging.info('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
logging.info('Beginning HOGE Downwash calculations...')
# To iterate on downwash solution, a function to calculate rotor thrust must
# be defined to take tip_angle as an input.
# Then, scipy.minimize can be used to reach zero error.

# TEMP: Inputs
delta_1 = -0.0216
delta_2 = 0.4
k_i = 1.1
CLIMB_RATE = 0   #[ft/min]

def thrust_generator(num: int,
                    TWIST: int,
                    theta_tip: float,
                    sol: float,
                    LIFT_CURVE_SLOPE: float,
                    v_tip: float,
                    CLIMB_RATE: float,
                    R: float,
                    B: float = 1.0
                   ) -> pd.DataFrame:
    '''
    Note: Despite the name, this function is not a pythonic generator.
    
    This function generates a thrust table for a rotor,
    using Blade-Element-Momentum Theory.
    
    This model assumes a constant-chord blade with no root cut-out,
    and a linear twist.
    '''
    df = pd.DataFrame(np.linspace(0.001, B, num), columns=['x'])
    df['theta'] = TWIST*(df.x - 1) + theta_tip

    c_1 = sol*LIFT_CURVE_SLOPE*v_tip/16
    c_2 = CLIMB_RATE/2 + sol*LIFT_CURVE_SLOPE*v_tip/16
    c_3 = 4*CLIMB_RATE**2/(sol*LIFT_CURVE_SLOPE*v_tip)

    # create a temporary variable which is a function of
    # blade stationand blade-element angle
    # used to calculate the final induced velocity.
    df['v_step'] = 2*(np.radians(df.theta)*df.x*v_tip - CLIMB_RATE)
    df['v_induced'] = c_2 * (-1 + np.sqrt(1 + df.v_step/(c_3 + CLIMB_RATE + c_1)))
    # Calculate the differential thrust of each blade element
    # Until I know a way to elegantly do this with pandas
    # convert everything to np arrays and loop through
    # TODO: cleanup code
    x_temp = np.array(df.x)
    x_temp = np.append(x_temp, 0)
    vi_temp = np.array(df.v_induced)
    vi_temp[-1] = 0
    df['v_induced'] = -vi_temp
    dT_temp = np.array([])
    for i, vi in enumerate(vi_temp):
        dT_temp = np.append(dT_temp, 2*math.pi*rho*(CLIMB_RATE+vi)*vi*x_temp[i]*R*R*(x_temp[i+1] - x_temp[i]) )


    df['dT'] = dT_temp
    
    return df


def T_err(theta, req, args):
    '''
    Calculates the thrust error for iteration.
    '''
    df = thrust_generator(num = args[0],
                          TWIST = args[1],
                          theta_tip = theta,
                          sol = args[2],
                          LIFT_CURVE_SLOPE= args[3],
                          v_tip = args[4],
                          CLIMB_RATE = args[5],
                          R = args[6],
                          B = args[7]
                         )
    
    calcd = df.dT.sum()
    err = abs(req-calcd)/req
    return err
    

###   Airfoil Lift factor correction from 2d to 3d   ###

# TODO: a_0 = 0.109662 ; a = a_0 / (1 + 57.3*a_0/(pi*AR))
# 0.109662 is the ideal, infinite airfoil with no edge effects.
# This equation transforms it into a 3d lift-curve-slope

# Aspect ratio is Rotor radius over the equivalent chord.
ASPECT_RATIO = R/(CHORD/12)

a_0 = 2*math.pi
a = a_0 / (1 + 57.3*a_0/(math.pi*ASPECT_RATIO))



###   Compressibility correction factor   ###

# TODO: 1st term of 3-term profile drag is for compressible flow only.
#       need to correct this to account for compressibilty of such a fast
#       moving rotor.
#       delta_0, the 1st drag term, is: d_0i/(sqrt(abs(Mach#**2 - 1)))
#       d_0i, the uncorrected 1st term, is given.

# The Mach num here is the mach @ 80% blade station = v_tip*0.8/c_sound
# c_sound is the local speed of sound, aka sqrt(gamma*R*T) = sqrt(1.4*1716.4*T), where T is in Rankine.
c_sound = math.sqrt(1.4*1716.4*T)
mach_08 = v_tip*0.8/c_sound
delta_0 = 0.0080/math.sqrt(abs(mach_08**2 - 1))


# TODO: Get Ct from Momentum theory = T/(rho*A*v_tip**2)
Ct_mom = THRUST/(rho*A*v_tip**2)

# TODO: Get B correction for tip-loss = 1 - (sqrt(2*Ct)/b)
B = 1 - math.sqrt(2*Ct_mom)/b

# TODO: ideal to linear twist correction
#       Power*correction = linear-twist power, based on ideal momentum theory calculations.
#       HP_i*k_i = HP_lin , k_i is given, usually 5-10% more (k_i=1.1)
#       HP_i = Power (converted to horsepower)
#       Power = C_p*rho*A*v_tip**3
#       C_p = C_q = 3-term drag model


C_qi = 0.5*Ct_mom*math.sqrt( (CLIMB_RATE/(OMEGA*R))**2 + 2*Ct_mom/B**2 )
C_qvi = CLIMB_RATE*Ct_mom/(2*OMEGA*R)
C_q0 = sol*delta_0/8
C_q1 = (2*delta_1/(3*a))*(Ct_mom/B**2)
C_q2 = (4*delta_2/(sol*a**2))*(Ct_mom/B**2)**2

C_q = C_qi + C_qvi + C_q0 + C_q1 + C_q2

P = C_q*rho*A*v_tip**3
HP_i = P/550   # Power, in horsepower
HP_i = HP_i
HP = HP_i * k_i
Q = HP*550*R/v_tip


logging.info('   {name:>18}: {val:>9.5f} [{unit:}]'.format(name='3D Lift Coef.', val=a, unit='cl/deg'))
logging.info('')
logging.info('Compressibility Correction: ')
logging.info('   {name:>18}: {val:>9.5f} [{unit:}]'.format(name='0.8STA Mach Num', val=mach_08, unit=''))
logging.info('   {name:>18}: {val:>9.5f} [{unit:}]'.format(name='delta_0', val=delta_0, unit=''))
logging.info('')
logging.info('According to pure Momentum Theory: ')
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='Coef. of Thrust', val=Ct_mom, unit=''))
logging.info('   {name:>18}: {val:>10.3f} [{unit:}]'.format(name='Tip-Loss Factor', val=B, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='C_qi', val=C_qi, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='C_qvi', val=C_qvi, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='C_q0', val=C_q0, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='C_q1', val=C_q1, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='C_q2', val=C_q2, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='Coef. of Pwr/Torq', val=C_q, unit=''))
logging.info('   {name:>18}: {val:>10.3E} [{unit:}]'.format(name='Total, Ideal Power', val=P, unit='ftlbs/s'))
logging.info('   {name:>18}: {val:>10.3f} [{unit:}]'.format(name='Corrected Power', val=HP, unit='hp'))
logging.info('   {name:>18}: {val:>10.3f} [{unit:}]'.format(name='Total Torque', val=Q, unit='lbft'))
logging.info('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
logging.info('')

 2020-04-08 19:37:12,856 -  INFO -  -.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
 2020-04-08 19:37:12,857 -  INFO -  Beginning HOGE Downwash calculations...
 2020-04-08 19:37:12,858 -  INFO -          3D Lift Coef.:   0.94124 [cl/deg]
 2020-04-08 19:37:12,859 -  INFO -  
 2020-04-08 19:37:12,859 -  INFO -  Compressibility Correction: 
 2020-04-08 19:37:12,860 -  INFO -        0.8STA Mach Num:   0.54174 []
 2020-04-08 19:37:12,861 -  INFO -                delta_0:   0.00952 []
 2020-04-08 19:37:12,861 -  INFO -  
 2020-04-08 19:37:12,862 -  INFO -  According to pure Momentum Theory: 
 2020-04-08 19:37:12,862 -  INFO -        Coef. of Thrust:  3.937E-03 []
 2020-04-08 19:37:12,864 -  INFO -        Tip-Loss Factor:      0.978 []
 2020-04-08 19:37:12,864 -  INFO -                   C_qi:  1.787E-04 []
 2020-04-08 19:37:12,865 -  INFO -                  C_qvi:  0.000E+00 []
 2020-04-08 19:37:12,865 -  INFO -                   C_q0:  7.502E-05 []
 2020-04-08 19:37:12,866 -  INFO -          

In [ ]:
## Note: With all the correction factors above, we have a roughly approximate solution
#        for realistic power, using just momentum theory. The blade-element portion
#        is not necessary yet.

#########################################
###   Blade-Element Momentum Theory   ###
###     Downwash Distribution         ###
###            (HOGE)                 ###
#########################################

args = (50, -12, sol, a, v_tip, CLIMB_RATE, R, 1)
res = scipy.optimize.minimize(T_err, x0=2, args=(THRUST, args), tol=0.01, options={'disp':True})
logging.info('  SciPy Optimization resulted in theta_tip = {:.3f}, after {} iterations.'.format(res.x[0], res.nfev))
logging.info('  Using optimization results to calculate Main Rotor data...')
logging.info('')
theta = res.x[0]
MR = thrust_generator(num = args[0],
                          TWIST = args[1],
                          theta_tip = theta,
                          sol = args[2],
                          LIFT_CURVE_SLOPE= args[3],
                          v_tip = args[4],
                          CLIMB_RATE = args[5],
                          R = args[6],
                          B = args[7]
                         )

logging.info(f'   Total Main Rotor Thrust = {MR.dT.sum():10.2f} [lb]')
logging.info('')
logging.info('Blade Station Downwash Table: ')
print(MR)

In [31]:
#######################################
###   Forward Flight Calculations   ###
#######################################
logging.info('Beginning Forward Flight Calculations...')

# Max Airspeed, in knots
VNE = 160
fe = 12.4     # Equivalent Flat-Plate Drag Area  [ft2]

df = pd.DataFrame(data=[20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 130, 140, 150, 160], columns=['Airspeed'])

# Dynamic Pressure = 1/2 rho V^2
df['q'] = 0.5*rho*(df.Airspeed * 1.68781)**2    # 1.689 is simply the conversion factor from kts to ft/s

# Advance Ratio
df['mu'] = df.Airspeed * 1.689 / v_tip

# Hover Induced Velocity
v_0 = math.sqrt(THRUST/(2*rho*math.pi*(B*R)**2))
# Induced velocity in Forward Flight, using Glauert's Model
df['v_if'] = v_0 * (-0.5*(df.Airspeed/v_0)**2 + np.sqrt((df.Airspeed/v_0)**4 / 4 + 1))

# Thrust coef. over solidity
df['Cts'] = THRUST/(rho*A*v_tip**2) / sol

# Blade loading
df['tc'] = 2*df.Cts

# Empirical lower bound of blade loading
df['tc_lower'] = -0.6885*df.Cts + 0.3555

# Change in Drag Coef. (due to retreating blade stall)
# From a NASA CR
df['F'] = ((df.Cts/(1-df.mu)**2) * (1 + fe*df.q/GW)) - 0.1376
df['del_cds'] = 18.3*(1-df.mu)**2*df.F**3
# This value should never be less than zero, so clip it.
df.del_cds.clip(lower=0, inplace=True)

# Mach Drag Divergence of the airfoil: At what Mach would shockwaves start to form?
df['MDD'] = 0.82 - 2.4*df.Cts

# MY90. Mach at the tip
df['MY90'] = (df.Airspeed*1.68781 + v_tip)/c_sound

# Change in Drag Coef. from compressibility
df['del_cdcomp'] = 0.2*(df.MY90 - df.MDD)**3 + 0.0085*(df.MY90 - df.MDD)

# Total Drag Coef.
df['cd'] = 0.0095 + df.del_cds + df.del_cdcomp

# Induced Horsepower
df['Hp_ind'] = THRUST*df.v_if/550

# Profile Horsepower
df['Hp_pro'] = sol*df.cd*(1+4.65*df.mu**2)*rho*math.pi*R**2*v_tip**3/4400

# Parasite Power
df['Hp_par'] = fe*rho*(df.Airspeed*1.68781)**3/1100

# Main Rotor Horsepower
df['MR_hp'] = df.Hp_ind + df.Hp_pro + df.Hp_par

# Main Rotor Torque
df['MR_Q'] = 5252*df.MR_hp/(OMEGA*60/(2*math.pi))

# Anti-torque Required from Tail Rotor Thrust
# MR Torque over the moment arm
df['T_at'] = df.MR_Q/(2*l_tail)

MR = df.copy()

# Copy the dataframe to a new variable, for Tail calculations
# Carry over the airspeed and anti-torque thrust
TR = pd.DataFrame(data=MR.Airspeed)
TR['T_at'] = MR.T_at.copy()
TR['q'] = MR.q.copy()

# Calculate the anti-torque provided
# by the vertical tail
# Lift = cl*wing_area*dynamic_pressure
TR['L_vt'] = cl_vt*S_vt*TR.q

# Anti-torque minus Vfin Lift
TR['TTR'] = TR.T_at - TR.L_vt

# Induced Drag from the Vfin
TR['D_vt'] = TR.L_vt**2/(2*TR.q*S_vt*AR_vt)

MR.set_index('Airspeed', inplace=True)
# logging.info('Vehicle Main Rotor Characteristics over the Speed Sweep: ')
MR

# logging.info('Vehicle Tail Rotor Characteristics over the Speed Sweep: ')
# TR

 2020-04-08 19:22:58,275 -  INFO -  Beginning Forward Flight Calculations...


,q,mu,v_if,Cts,tc,tc_lower,F,del_cds,MDD,MY90,del_cdcomp,cd,Hp_ind,Hp_pro,Hp_par,MR_hp,MR_Q,T_at
Airspeed,,,,,,,,,,,,,,,,,,
20,1.355207,0.044683,28.966576,0.062443,0.124886,0.312508,-0.068949,0.000000e+00,0.670136,0.707415,0.000327,0.009827,271.232480,140.551598,1.031379,412.815457,5255.639141,123.895312
30,3.049216,0.067024,23.609563,0.062443,0.124886,0.312508,-0.065320,0.000000e+00,0.670136,0.722533,0.000474,0.009974,221.071365,144.293054,3.480904,368.845323,4695.846251,110.698874
40,5.420828,0.089365,18.160444,0.062443,0.124886,0.312508,-0.061287,0.000000e+00,0.670136,0.737652,0.000635,0.010135,170.047792,148.959802,8.251031,327.258624,4166.397371,98.217760
50,8.470044,0.111706,13.607419,0.062443,0.124886,0.312508,-0.056802,0.000000e+00,0.670136,0.752770,0.000815,0.010315,127.414928,154.655765,16.115294,298.185987,3796.267606,89.492400
60,12.196864,0.134048,10.218932,0.062443,0.124886,0.312508,-0.051810,0.000000e+00,0.670136,0.767888,0.001018,0.010518,95.686363,161.496542,27.847229,285.030134,3628.777708,85.544029
70,16.601287,0.156389,7.811661,0.062443,0.124886,0.312508,-0.046247,0.000000e+00,0.670136,0.783007,0.001247,0.010747,73.145551,169.612129,44.220367,286.978048,3653.577006,86.128642
80,21.683314,0.178730,6.107920,0.062443,0.124886,0.312508,-0.040042,0.000000e+00,0.670136,0.798125,0.001507,0.011007,57.192342,179.149652,66.008245,302.350240,3849.283567,90.742187
90,27.442944,0.201071,4.883028,0.062443,0.124886,0.312508,-0.033113,0.000000e+00,0.670136,0.813243,0.001803,0.011303,45.722895,190.276093,93.984396,329.983384,4201.086860,99.035522
100,33.880177,0.223413,3.982636,0.062443,0.124886,0.312508,-0.025361,0.000000e+00,0.670136,0.828362,0.002137,0.011637,37.291959,203.181015,128.922354,369.395329,4702.848493,110.863944


In [36]:
a_0

0.109662